In [2]:
import os
import re
from collections import defaultdict

import pandas as pd

import camelot

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
dir_item = '/home/jake/Documents/News_Item/'
dir_proj = dir_item + 'fukushima/'
dir_tableau = dir_proj + 'tableau_data/'

dir_pdf = './pdf/'
dir_tables = './tables/'

if not os.path.exists(dir_pdf):
    os.makedirs(dir_pdf)

if not os.path.exists(dir_tables):
    os.makedirs(dir_tables)

In [6]:
dirs = [dir_pdf + f for f in sorted(os.listdir(dir_pdf)) if not f.startswith('.')]
dirs[:3]

['./pdf/001', './pdf/002', './pdf/003']

In [7]:
files = []

for d in dirs:
    files.append([d + f for f in sorted(os.listdir(d))])
    break

files

[['./pdf/001201900820+기타국가+방사능+검출+현황.pdf',
  './pdf/00120190816+기타국가+방사능+검출+현황.pdf',
  './pdf/00120190819+기타국가+방사능+검출+현황.pdf',
  './pdf/00120190819+일본산+수입식품+검사현황+일일보고.pdf',
  './pdf/00120190820+일본산+수입식품+검사현황+일일보고.pdf',
  './pdf/001일본산+및+기타국가산+수입수산물+방사능검사+현황(8월+19일+완료건).pdf',
  './pdf/001일본산+및+기타국가산+수입수산물+방사능검사+현황(8월+20일+완료건).pdf',
  './pdf/001일본산+및+기타국가산+수입수산물+방사능검사+현황(8월+21일+완료건).pdf',
  './pdf/001일본산+수입축산물+방사능검사+현황(2019.8.20).pdf',
  './pdf/001일본산+수입축산물+방사능검사+현황(2019.8.21).pdf']]

In [191]:
def get_classified_files(files):
    '''
    group A: 수입식품
    group A2: 수입식품 (2016)
    group B: 수산물
    group C: 축산물
    group D: 축수산물
    group E: 나머지
    '''
    U = set(files)
#     A = set([f for f in files if ('일본' in f) and ('수입식품' in f)])
    A = set([f for f in files if re.match(r'\d\d\d\/\d+.*\.csv', f)])
    # 348/20150129-page-1-table-1.csv
    print(len(A))
#     print(files[0])
#     A = A | A2
    B = set([f for f in files if ('일본' in f) and ('수산물' in f)])
    C = set([f for f in files if ('일본' in f) and ('축산물' in f)])
    D = set([f for f in files if ('일본' in f) and ('축수산물' in f)])
    E = U - A - B - C - D
    if len(U) != len(A | B | C | D | E):
        print("E: ", E)
        print("A & B: ", len(A & B))
        print("A & C: ", len(A & C))
        print("A & D: ", len(A & D))
        print("B & C: ", len(B & C))
        print("B & D: ", len(B & D))
        print("C & D: ", len(C & D))
        print("C & E: ", len(C & E))
        print("D & E: ", len(D & E))
        print("A+B+C+D+E: ", len(A | B | C | D | E))
        print("Total: ", len(files))
    return A, B, C, D, E

In [192]:
def get_imported():
    imported = defaultdict(list)

    # page = 21
    for page in range(1, 430):
        dir_page = f'{page:>03}'
        try:
            files = sorted(os.listdir(dir_tables + dir_page))
        except FileNotFoundError as e:
            continue
        if not files:
            continue
        A, B, C, D, E = get_classified_files([dir_page+'/'+f for f in files])
        imported['수입식품'].extend(A)
        imported['수산물'].extend(B)
        imported['축산물'].extend(C)
        imported['축수산물'].extend(D)
        imported['나머지'].extend(E)
    return imported


imported = get_imported()

13
18
22
18
16
18
16
17
18
10
19
21
14
27
21
22
17
19
17
18
17
11
22
16
13
14
21
24
17
16
14
24
14
25
12
17
18
19
26
24
17
20
19
36
20
18
18
18
17
16
32
15
21
28
13
29
14
23
37
8
28
21
7
20
27
16
22
34
15
9
45
1
19
21
19
23
13
15
19
19
14
21
18
17
15
17
18
17
24
13
17
20
19
21
15
21
21
13
19
21
19
16
22
14
26
16
13
21
19
19
11
20
13
23
18
18
22
12
28
27
24
14
19
21
20
14
23
13
15
15
17
20
21
17
28
17
13
28
7
25
19
23
16
17
18
20
20
10
24
23
34
16
0
27
31
27
5
23
27
19
16
14
21
14
22
17
30
19
33
21
35
27
39
44
46
58
52
36
48
50
38
34
38
16
1
46
7
5
15
33
14
19
22
16
21
5
16
16
4
30
5
43
6
6
26
13
21
12
22
7
12
5
16
18
13
19
12
23
14
17
16
16
27
10
18
12
16
22
17
14
14
14
22
15
13
15
15
20
10
24
15
13
12
17
16
21
14
22
16
15
14
25
22
8
17
16
15
19
11
17
20
20
16
16
13
24
20
12
19
16
25
16
13
12
18
16
16
12
21
12
14
22
17
15
16
29
10
20
15
14
15
17
24
15
22
15
19
10
23
17
17
10
12
20
22
9
16
16
15
19
14
15
20
15
16
12
14
18
17
23
22
11
19
18
18
11
27
14
9
18
0
17
11
23
24
20
12
21
16
13
1

In [169]:
def get_general_or_fish(kind='수입식품'):
    '''
    수입식품, 수산물, 축산물
    '''
    file = [dir_tables + r for r in imported[kind]]
    result = pd.DataFrame()
    
    # fix later
    for f in file:
        tmp = pd.read_csv(f)
        tmp['file'] = f
        if tmp.shape[1] == 9:
            tmp.columns = ['순번', '품목명', '제조·생산지', '제조일자(유통기한)', '수입중량(kg)', '세슘', '요오드', '판정', 'file']
            result = result.append(tmp)

    result.drop('순번', axis=1, inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result

In [115]:
tmp = [dir_tables + r for r in imported['수입식품']]
tmp2 = []
for t in tmp:
    df = pd.read_csv(t)
    df['file'] = t
    if df.shape[1] != 9:
        print(df.shape)
        tmp2.append(df)

(7, 12)
(7, 12)
(7, 12)
(2, 7)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(2, 7)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7, 12)
(7

In [127]:
shapes = set([t.shape for t in tmp2])
shapes

{(1, 7), (2, 2), (2, 7), (3, 2), (7, 12)}

In [130]:
two = [t for t in tmp2 if t.shape[1]==2]
two[0]
tmp3 = pd.DataFrame()

for s in two:
#     s.columns = ['순번', '품목명', '제조·생산지', '제조일자(유통기한)', '수입중량(kg)', '검사결과', 'file']
    tmp3 = tmp3.append(s)
    
print(tmp3.shape)
tmp3

(15, 2)


,검사결과 수입 제조일자 순번 품목명 제조·생산지 중량 세슘 요오드 (유통기한) 판정 (kg) (기준:100Bq/kg) (기준:300Bq/kg),file
0,NaN,./tables/074/20180919+일본산+수입식품+검사현황+일일보고-page-6-table-1.csv
1,147\n기타주류\n효고현\n2018-08-08\n18\n적합\n불검출\n불검출,./tables/074/20180919+일본산+수입식품+검사현황+일일보고-page-6-table-1.csv
2,148\n청주\n효고현\n2018-07-20\n18\n적합\n불검출\n불검출,./tables/074/20180919+일본산+수입식품+검사현황+일일보고-page-6-table-1.csv
0,NaN,./tables/099/20180605+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv
1,"120\n효모식품\n오이타현\n2018-04-22\n6,000\n불검출\n불검출\n적합",./tables/099/20180605+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv
2,121\n혼합제제\n미에현\n2018-03-12\n108\n불검출\n불검출\n적합,./tables/099/20180605+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv
0,NaN,./tables/105/20180510+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv
1,80\n혼합제제\n오사카부\n2018-04-11\n210\n적합\n불검출\n불검출,./tables/105/20180510+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv
0,NaN,./tables/117/20180322+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv
1,80\n탄산음료\n오사카부\n2018-02-02\n900\n적합\n불검출\n불검출,./tables/117/20180322+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv


In [125]:
seven = [t for t in tmp2 if t.shape[1]==7]
seven[0]
tmp3 = pd.DataFrame()

for s in seven:
    s.columns = ['순번', '품목명', '제조·생산지', '제조일자(유통기한)', '수입중량(kg)', '검사결과', 'file']
    tmp3 = tmp3.append(s)
    
print(tmp3.shape)
tmp3.head()

(184, 7)


,순번,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),검사결과,file
0,NaN,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:100Bq/kg),./tables/002/20190814+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv
1,121.0,소스,히로시마현,2019-05-29,76,불검출\n불검출\n적합,./tables/002/20190814+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv
0,NaN,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:100Bq/kg),./tables/019/20190531+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv
1,120.0,덱스트린,효고현,2019-04-23,"8,000",불검출\n불검출\n적합,./tables/019/20190531+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv
0,NaN,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:300Bq/kg),./tables/044/20190213+일본산+수입식품+검사현황+일일보고-page-7-table-1.csv


In [126]:
tmp3

,순번,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),검사결과,file
0,NaN,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:100Bq/kg),./tables/002/20190814+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv
1,121.0,소스,히로시마현,2019-05-29,76,불검출\n불검출\n적합,./tables/002/20190814+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv
0,NaN,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:100Bq/kg),./tables/019/20190531+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv
1,120.0,덱스트린,효고현,2019-04-23,"8,000",불검출\n불검출\n적합,./tables/019/20190531+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv
0,NaN,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:300Bq/kg),./tables/044/20190213+일본산+수입식품+검사현황+일일보고-page-7-table-1.csv
1,190.0,소스,히로시마현,2018-12-14,522,불검출\n불검출\n적합,./tables/044/20190213+일본산+수입식품+검사현황+일일보고-page-7-table-1.csv
0,NaN,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:300Bq/kg),./tables/051/20190107+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv
1,120.0,소스,후쿠시마현,2018-09-21,160,불검출\n불검출\n적합,./tables/051/20190107+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv
0,NaN,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:300Bq/kg),./tables/053/20181227+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv
1,120.0,맥주,후쿠오카현,2018-12-04,"52,344",불검출\n불검출\n적합,./tables/053/20181227+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv


In [122]:
tmp[:4]

['./tables/001/20190820+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv',
 './tables/001/20190819+일본산+수입식품+검사현황+일일보고-page-1-table-2.csv',
 './tables/001/20190819+일본산+수입식품+검사현황+일일보고-page-1-table-1.csv',
 './tables/001/20190819+일본산+수입식품+검사현황+일일보고-page-2-table-1.csv']

In [116]:
len(tmp2)

747

In [120]:
tmp2[1]

,품목,계,Unnamed: 2,농산물,Unnamed: 4,가공식품,Unnamed: 6,건강기능식품,Unnamed: 8,식품첨가물,Unnamed: 10,file
0,처리\n일자,건수\n중량(㎏),NaN,건수\n중량(㎏),NaN,건수\n중량(㎏),NaN,건수\n중량(㎏) 건수,NaN,중량(㎏),NaN,./tables/001/20190820+일본산+수입식품+검사현황+일일보고-page-1-table-1.csv
1,합계,226,"1,064,554",1,"17,499",168,"984,556",2,"6,295",55,"56,204",./tables/001/20190820+일본산+수입식품+검사현황+일일보고-page-1-table-1.csv
2,8.19,117,"896,464",0,0,94,"848,709",1,633,22,"47,122",./tables/001/20190820+일본산+수입식품+검사현황+일일보고-page-1-table-1.csv
3,8.20,109,"168,090",1,"17,499",74,"135,847",1,"5,662",33,"9,082",./tables/001/20190820+일본산+수입식품+검사현황+일일보고-page-1-table-1.csv
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,./tables/001/20190820+일본산+수입식품+검사현황+일일보고-page-1-table-1.csv
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,./tables/001/20190820+일본산+수입식품+검사현황+일일보고-page-1-table-1.csv
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,./tables/001/20190820+일본산+수입식품+검사현황+일일보고-page-1-table-1.csv


In [119]:
tmp2[2]

,품목,계,Unnamed: 2,농산물,Unnamed: 4,가공식품,Unnamed: 6,건강기능식품,Unnamed: 8,식품첨가물,Unnamed: 10,file
0,처리\n일자,건수\n중량(㎏),NaN,건수\n중량(㎏),NaN,건수\n중량(㎏),NaN,건수\n중량(㎏),NaN,건수\n중량(㎏),NaN,./tables/002/20190816+일본산+수입식품+검사현황+일일보고-page-1-table-1.csv
1,합계,448,"2,302,860",3,"4,140",299,"2,226,445",3,"1,029",143,"71,246",./tables/002/20190816+일본산+수입식품+검사현황+일일보고-page-1-table-1.csv
2,8.12,117,"699,592",1,30,62,"684,253",0,0,54,"15,309",./tables/002/20190816+일본산+수입식품+검사현황+일일보고-page-1-table-1.csv
3,8.13,89,"402,242",0,0,65,"377,604",2,"1,020",22,"23,618",./tables/002/20190816+일본산+수입식품+검사현황+일일보고-page-1-table-1.csv
4,8.14,121,"1,001,645",2,"4,110",89,"974,721",0,0,30,"22,814",./tables/002/20190816+일본산+수입식품+검사현황+일일보고-page-1-table-1.csv
5,8.16,121,"199,381",0,0,83,"189,867",1,9,37,"9,505",./tables/002/20190816+일본산+수입식품+검사현황+일일보고-page-1-table-1.csv
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,./tables/002/20190816+일본산+수입식품+검사현황+일일보고-page-1-table-1.csv


In [121]:
tmp2[3]

,순번,품목명,제조·생산지,제조일자 (유통기한),수입 중량 (kg),검사결과,file
0,NaN,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:100Bq/kg),./tables/002/20190814+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv
1,121.0,소스,히로시마현,2019-05-29,76.0,불검출\n불검출\n적합,./tables/002/20190814+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv


In [118]:
tmp2[-1]

,품목,계,Unnamed: 2,농산물,Unnamed: 4,가공식품,Unnamed: 6,건강기능식품,Unnamed: 8,식품첨가물,Unnamed: 10,file
0,접수\n일자,건수 중량(㎏) 건수 중량(㎏) 건수 중량(㎏) 건수 중량(㎏) 건수,NaN,NaN,NaN,NaN,NaN,NaN,NaN,중량(㎏),NaN,./tables/316/일본산+수입식품+검사현황(0708)-page-1-table-1.csv
1,합계,302,"1,038,364",0.0,0.0,209.0,"971,481",5.0,"1,076",88,"65,807",./tables/316/일본산+수입식품+검사현황(0708)-page-1-table-1.csv
2,7.6,117,"196,401",0.0,0.0,71.0,"186,274",3.0,69,43,"10,058",./tables/316/일본산+수입식품+검사현황(0708)-page-1-table-1.csv
3,7.7,101,"424,692",0.0,0.0,71.0,"394,055",1.0,27,29,"30,611",./tables/316/일본산+수입식품+검사현황(0708)-page-1-table-1.csv
4,7.8,84,"417,271",0.0,0.0,67.0,"391,152",1.0,980,16,"25,138",./tables/316/일본산+수입식품+검사현황(0708)-page-1-table-1.csv
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,./tables/316/일본산+수입식품+검사현황(0708)-page-1-table-1.csv
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,./tables/316/일본산+수입식품+검사현황(0708)-page-1-table-1.csv


# 나머지

In [102]:
etc = get_general_or_fish('나머지')
print(etc.shape)
etc.head()

(84243, 8)


,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file
0,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:100Bq/kg),NaN,NaN,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
1,NaN,NaN,NaN,"2,330",NaN,NaN,NaN,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
2,고형차,러시아연방,2022.07.31.,100,55,불검출,적합,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
3,능이/건조,중국,(2019.08.01.),"2,230",3,-,적합,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
4,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:100Bq/kg),NaN,NaN,./tables/002/201900813+기타국가+방사능+검출+현황-page-1-table-1.csv


In [103]:
etc['제조·생산지'].value_counts()

효고현                    7553
아이치현                   5576
오사카부                   4749
사이타마현                  4114
니가타현                   3070
훗카이도                   3047
교토부                    3014
후쿠오카현                  2798
히로시마현                  2738
시즈오카현                  2656
나가노현                   2469
가나가와현                  2404
미에현                    2313
이바라키현                  2123
지바현                    2023
러시아                    1744
도치기현                   1672
오카야마현                  1617
시가현                    1332
도쿄도                    1262
대만                     1153
에히메현                   1119
군마현                    992 
오이타현                   940 
와카야마현                  936 
사가현                    915 
구마모토현                  911 
나라현                    897 
시마네현                   855 
나가사키현                  805 
가고시마현                  788 
-                      671 
야마가타현                  634 
기후현                    572 
도쿠시마현                  537 
야마구치현               

In [105]:
etc_japan = etc.loc[etc['제조·생산지'].notna() & etc['제조·생산지'].str.endswith('현')]
etc_japan.head()

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file
319,혼합제제,미에현,2018-10-18,270,불검출,불검출,적합,./tables/056/20181211+일일보고-page-7-table-1.csv
320,폴리글리세린지방산\n에스테르,미에현,2018-09-25,15,불검출,불검출,적합,./tables/056/20181211+일일보고-page-7-table-1.csv
321,자당지방산에스테르,기후현,2018-08-30,720,불검출,불검출,적합,./tables/056/20181211+일일보고-page-7-table-1.csv
322,자당지방산에스테르,미에현,2018-10-10,"3,600",불검출,불검출,적합,./tables/056/20181211+일일보고-page-7-table-1.csv
323,자당지방산에스테르,나가노현,2018-09-06,200,불검출,불검출,적합,./tables/056/20181211+일일보고-page-7-table-1.csv


In [107]:
etc_japan.loc[etc_japan['세슘']=='2']

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file
74575,볶은커피*,사이타마현,2014-05-30,23.04,2,불검출,적합,./tables/380/20140826_방사능_검사_현황-page-3-table-1.csv


# 축수산물

In [97]:
mix = get_general_or_fish('축수산물')
print(mix.shape)
mix.head()

(17677, 8)


,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file
0,NaN,NaN,NaN,NaN,세슘\n(기준:100Bq/Kg),요오드\n(기준:300Bq/Kg),판정,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv
1,NaN,-,-,"23,876",-,-,-,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv
2,활돔(참돔),에히메현,2017-03-17,4672,불검출,불검출,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv
3,냉장명태,홋카이도,2017-03-17,1157,불검출,불검출,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv
4,냉장명태,홋카이도,2017-03-17,1280,불검출,불검출,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv


In [98]:
mix['세슘'].value_counts()

불검출                  15205
-                    1033 
세슘\n(기준:100Bq/Kg)    895  
세슘\n(기준:50Bq/Kg)     462  
없                    1    
Name: 세슘, dtype: int64

In [100]:
mix.loc[mix['세슘']=='-'].head()

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file
1,NaN,-,-,"23,876",-,-,-,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv
13,NaN,-,-,96,-,-,-,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-2-table-1.csv
16,NaN,-,-,180,-,-,-,./tables/185/일본산_수입축수산물_방사능검사_현황(3월14일_완료건)-page-3-table-1.csv
41,NaN,-,-,420,-,-,-,./tables/185/일본산_수입축수산물_방사능검사_현황(3월13일_완료건)-page-2-table-2.csv
45,NaN,-,-,0,-,-,-,./tables/185/일본산_수입축수산물_방사능검사_현황(3월16일_완료건)-page-3-table-1.csv


In [99]:
mix['요오드'].value_counts()

불검출                   15205
-                     1033 
요오드\n(기준:300Bq/Kg)    895  
요오드\n(기준:100Bq/Kg)    462  
음                     1    
Name: 요오드, dtype: int64

In [101]:
mix['판정'].value_counts()

적합    15205
판정    1357 
-     1033 
Name: 판정, dtype: int64

# 축산물

In [89]:
meat = get_general_or_fish('축산물')
meat.head()

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file
0,NaN,NaN,NaN,NaN,세슘\n(기준:\n50Bq/Kg),요오드\n(기준:\n100Bq/Kg),판정,./tables/001/일본산+수입축산물+방사능검사+현황(2019.8.20)-page-1-table-1.csv
1,NaN,-,-,"6,516",-,-,-,./tables/001/일본산+수입축산물+방사능검사+현황(2019.8.20)-page-1-table-1.csv
2,가공치즈,사이타마현,2019-08-20,810,불검출,불검출,적합,./tables/001/일본산+수입축산물+방사능검사+현황(2019.8.20)-page-1-table-1.csv
3,가공치즈,사이타마현,2019-08-20,"1,008",불검출,불검출,적합,./tables/001/일본산+수입축산물+방사능검사+현황(2019.8.20)-page-1-table-1.csv
4,아이스크림,훗카이도,2019-08-20,342,불검출,불검출,적합,./tables/001/일본산+수입축산물+방사능검사+현황(2019.8.20)-page-1-table-1.csv


In [92]:
print(meat.shape)
meat['세슘'].value_counts()

(465, 8)


불검출                    280
-                      92 
세슘\n(기준:\n50Bq/Kg)     88 
세슘\n(기준:\n100Bq/Kg)    4  
불검출*                   1  
Name: 세슘, dtype: int64

In [94]:
meat.loc[meat['세슘']=='불검출*']

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file
128,식육추출가공품,야마가타현,2019-07-03,110,불검출*,불검출**,적합,./tables/012/일본산+수입축산물+방사능검사+현황(2019.7.3)-page-1-table-1.csv


In [93]:
meat['요오드'].value_counts()

불검출                     280
-                       92 
요오드\n(기준:\n100Bq/Kg)    88 
요오드\n(기준:\n300Bq/Kg)    4  
불검출**                   1  
Name: 요오드, dtype: int64

# 수산물

In [65]:
fish = get_general_or_fish(kind='수산물')
fish.head()
# def get_fish():
# fish = [dir_tables + r for r in imported['수산물']]
# result = pd.DataFrame()

# for f in fish:
#     tmp = pd.read_csv(f)
#     if tmp.shape[1] != 8:
#         print(tmp.shape)
#         print(tmp.head())

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file
0,NaN,NaN,NaN,NaN,세슘\n(기준:100Bq/Kg),요오드\n(기준:300Bq/Kg),판정,./tables/001/일본산+및+기타국가산+수입수산물+방사능검사+현황(8월+21일+완료건)-page-1-table-1.csv
1,활바리(능성어),에히메현,2019-08-21,831,불검출,불검출,적합,./tables/001/일본산+및+기타국가산+수입수산물+방사능검사+현황(8월+21일+완료건)-page-1-table-1.csv
2,활돔(참돔),에히메현,2019-08-21,"11,216",불검출,불검출,적합,./tables/001/일본산+및+기타국가산+수입수산물+방사능검사+현황(8월+21일+완료건)-page-1-table-1.csv
3,활우렁쉥이,홋카이도,2019-08-21,"8,789",불검출,불검출,적합,./tables/001/일본산+및+기타국가산+수입수산물+방사능검사+현황(8월+21일+완료건)-page-1-table-1.csv
4,활장어(먹장어),나가사키현,2019-08-21,"1,544",불검출,불검출,적합,./tables/001/일본산+및+기타국가산+수입수산물+방사능검사+현황(8월+21일+완료건)-page-1-table-1.csv


In [85]:
fish.tail(20)

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file
39773,"냉동다랑어(눈다랑어,목살)",시즈오카현,2014-4-2,"2,556",불검출,불검출,적합,./tables/410/일본산_수입축수산물_방사능검사_현황(4.2일_완료건)-page-1-table-1.csv
39774,"냉동다랑어(눈다랑어,F,횟감)",시즈오카현,2014-4-2,262,불검출,불검출,적합,./tables/410/일본산_수입축수산물_방사능검사_현황(4.2일_완료건)-page-1-table-1.csv
39775,냉동해삼(자숙),나가사키현,2014-4-2,120,불검출,불검출,적합,./tables/410/일본산_수입축수산물_방사능검사_현황(4.2일_완료건)-page-1-table-1.csv
39776,냉장돔(황돔),나가사키현,2014-4-2,260,불검출,불검출,적합,./tables/410/일본산_수입축수산물_방사능검사_현황(4.2일_완료건)-page-1-table-1.csv
39777,"냉동다랑어(눈다랑어,F,횟감)",시즈오카현,2014-4-2,"1,340",불검출,불검출,적합,./tables/410/일본산_수입축수산물_방사능검사_현황(4.2일_완료건)-page-1-table-1.csv
39778,"냉동다랑어(눈다랑어,목살)",시즈오카현,2014-4-2,563,불검출,불검출,적합,./tables/410/일본산_수입축수산물_방사능검사_현황(4.2일_완료건)-page-1-table-1.csv
39779,냉동다랑어(남방참다랑어),시즈오카현,2014-4-2,"3,555",불검출,불검출,적합,./tables/410/일본산_수입축수산물_방사능검사_현황(4.2일_완료건)-page-1-table-1.csv
39780,활가리비,홋카이도,2014-4-2,"7,155",불검출,불검출,적합,./tables/410/일본산_수입축수산물_방사능검사_현황(4.2일_완료건)-page-1-table-1.csv
39781,냉장홍어,야마구치현,2014-4-2,230,불검출,불검출,적합,./tables/410/일본산_수입축수산물_방사능검사_현황(4.2일_완료건)-page-1-table-1.csv
39782,냉장민어(수조기),야마구치현,2014-4-2,93,불검출,불검출,적합,./tables/410/일본산_수입축수산물_방사능검사_현황(4.2일_완료건)-page-1-table-1.csv


In [66]:
fish['세슘'].value_counts()

불검출                  31254
적합                   3218 
세슘\n(기준:100Bq/Kg)    2610 
-                    2150 
세슘\n(기준:50Bq/Kg)     462  
2                    2    
없                    1    
적합(1)                1    
Name: 세슘, dtype: int64

In [67]:
fish.loc[fish['세슘']=='적합'].head()

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file
4723,냉장명태,홋카이도,2019-01-10,"2,770",적합,적합,적합,./tables/050/일본산+및+기타국가산+수입수산물+방사능검사+현황(1월10일+완료건)-page-1-table-1.csv
4724,냉동가오리,홋카이도,2019-01-10,"22,600",적합,적합,적합,./tables/050/일본산+및+기타국가산+수입수산물+방사능검사+현황(1월10일+완료건)-page-1-table-1.csv
4725,활장어(먹장어),나가사키현,2019-01-10,"2,300",적합,적합,적합,./tables/050/일본산+및+기타국가산+수입수산물+방사능검사+현황(1월10일+완료건)-page-1-table-1.csv
4726,냉장홍어,홋카이도,2019-01-10,793,적합,적합,적합,./tables/050/일본산+및+기타국가산+수입수산물+방사능검사+현황(1월10일+완료건)-page-1-table-1.csv
4727,냉장홍어,야마구치현,2019-01-10,216,적합,적합,적합,./tables/050/일본산+및+기타국가산+수입수산물+방사능검사+현황(1월10일+완료건)-page-1-table-1.csv


In [69]:
fish.loc[fish['세슘'].notna() & fish['세슘'].str.isnumeric()].head()

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file
10971,냉동상어(청상아리),대만,2018-04-24,"2,747",2,불검출,적합,./tables/109/일본산+및+기타국가산+수입수산물+방사능검사+현황(4월24일+완료건)-page-2-table-2.csv
11384,냉동상어(청상아리),대만,2018-04-06,"27,460",2,불검출,적합,./tables/113/일본산+및+기타국가산+수입수산물+방사능검사+현황(4월6일+완료건)+수동-page-2-table-2.csv


In [71]:
fish['요오드'].value_counts()

불검출                   31256
적합                    3219 
요오드\n(기준:300Bq/Kg)    2610 
-                     2150 
요오드\n(기준:100Bq/Kg)    462  
음                     1    
Name: 요오드, dtype: int64

In [70]:
fish.loc[fish['요오드'].notna() & fish['요오드'].str.isnumeric()].head()

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file


In [72]:
fish.loc[fish['요오드']=='음'].head()

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file
27879,검,사,실,적,없,음,NaN,./tables/231/일본산_수입축수산물_방사능검사_현황(8월17일_완료건)-page-2-table-2.csv


# 수입식품

In [193]:
general = get_general_or_fish('수입식품')
print(general.shape)
general.head()

(148473, 8)


,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file
0,NaN,NaN,NaN,NaN,세슘\n(기준:100Bq/kg),요오드\n(기준:100Bq/kg),판정,./tables/001/20190820+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv
1,과자,훗카이도,2019-07-24,23,불검출,불검출,적합,./tables/001/20190820+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv
2,화이트초콜릿,훗카이도,2019-07-24,41,불검출,불검출,적합,./tables/001/20190820+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv
3,청주,와카야마현,2019-07-22,907,불검출,불검출,적합,./tables/001/20190820+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv
4,초콜릿가공품,훗카이도,2019-07-24,41,불검출,불검출,적합,./tables/001/20190820+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv


In [194]:
general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148473 entries, 0 to 148472
Data columns (total 8 columns):
품목명           140641 non-null object
제조·생산지        141125 non-null object
제조일자(유통기한)    139362 non-null object
수입중량(kg)      142515 non-null object
세슘            145052 non-null object
요오드           142067 non-null object
판정            142059 non-null object
file          148473 non-null object
dtypes: object(8)
memory usage: 9.1+ MB


In [195]:
general.loc[general['file'].str.contains('2014')].shape

(17728, 8)

In [196]:
general.loc[general['file'].str.contains('2015')].shape

(24146, 8)

In [197]:
general.loc[general['file'].str.contains('2016')].shape

(25801, 8)

In [198]:
general.loc[general['file'].str.contains('2017')].shape

(28083, 8)

In [199]:
general.loc[general['file'].str.contains('2018')].shape

(32348, 8)

In [200]:
general.loc[general['file'].str.contains('2019')].shape

(20367, 8)

In [201]:
general['세슘'].value_counts()

불검출                                                         139913
세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:300Bq/kg)                   2868  
세슘\n(기준:100Bq/kg)                                           1652  
세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:100Bq/kg)                   119   
2                                                           84    
3                                                           65    
1                                                           58    
4                                                           42    
5                                                           14    
7                                                           11    
17                                                          9     
12                                                          9     
8                                                           8     
6                                                           8     
9                                                           7 

In [202]:
general.loc[general['세슘'].notna() & general['세슘'].str.isnumeric()]

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file
44,고형차,러시아연방,2022.07.31.,100,55,불검출,적합,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
45,능이/건조,중국,(2019.08.01.),"2,230",3,-,적합,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
424,능이버섯/건조,중국,2020.10.31.,429,13,-,적합,./tables/002/201900813+기타국가+방사능+검출+현황-page-1-table-1.csv
607,잼,스웨덴,2020.10.27.,806,1,불검출,적합,./tables/003/20190809+기타국가+방사능+검출+현황-page-1-table-1.csv
608,차가버섯/건조,러시아연방,(2019.6.13.),"3,520",18,불검출,적합,./tables/003/20190809+기타국가+방사능+검출+현황-page-1-table-1.csv
609,능이버섯/건조,중국,2022.01.25.,"1,260",2,-,적합,./tables/003/20190809+기타국가+방사능+검출+현황-page-1-table-1.csv
610,능이버섯/신선,중국,생산년도\n(2019년),21,6,-,적합,./tables/003/20190809+기타국가+방사능+검출+현황-page-1-table-1.csv
1227,능이/건조,중국,(2019.7.25.),664,2,-,적합,./tables/004/20190806+기타국가+방사능+검출+현황-page-1-table-1.csv
1471,능이/냉동,중국,(2019.7.7.),"14,418",4,-,적합,./tables/005/20190801_기타국가_방사능_검출_현황-page-1-table-1.csv
1472,능이/건조,중국,(2019.7.20.),400,1,-,적합,./tables/005/20190801_기타국가_방사능_검출_현황-page-1-table-1.csv


In [131]:
general.loc[general['file'].str.contains('2016')]

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,file


In [81]:
general['요오드'].value_counts()

불검출                     65855
요오드\n(기준:300Bq/kg)      617  
요오드\n(기준:100Bq/kg)      273  
요오드\n(기준:300Bq/k\ng)    2    
Name: 요오드, dtype: int64

In [36]:
general['판정'].value_counts()

적합     65836
판정     892  
취하     9    
취하*    2    
Name: 판정, dtype: int64